<a href="https://colab.research.google.com/github/edurso/tfod-wkspc/blob/master/tensorflow-object-detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup Workspace

In [ ]:
%%bash

# install deps 
python --version
pip install -U --pre tensorflow=="2.*"
pip install tf_slim pycocotools

# clear previous runs
rm -rf /tf/models/ /tf/workspace/
mkdir /tf

# clone repos
git clone --depth 1 https://github.com/tensorflow/models /tf/models
git clone --depth 1 https://github.com/edurso/tfod-wkspc /tf/workspace

# compile protobufs
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.

# install object detector package
cp object_detection/packages/tf2/setup.py . 
pip install .

# test installation
cd /content/models/research/object_detection/builders/
python model_builder_tf2_test.py

In [ ]:
from google.colab import drive
drive.mount('/tf/data')
DATA='/tf/data/MyDrive/tensorflow/power-port-targeting/'
MODEL='centernet_mobilenetv2_fpn_od'

Now, go to the [model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) and copy a model to `pre-trained-models/`. 

Split the dataset and copy images to `images/test/` and `images/train/`.

Add `label_map.pbtxt` to `annotations/`.

Make a new directory in `models/` named similarly (or the same) to the model you downloaded from the zoo. Copy the `pipeline.config` file from the model you downloaded to this directory.

Verify the colab runtime is configured for a GPU.

Now update the `pipeline.config` in `models/` to refelect your dataset.

In [ ]:
%cd '/tf/data/MyDrive/tensorflow/power-port-targeting/'

!export SCRIPTS="/tf/workspace/scripts"

# Generate TFRecord for Training Data
!python $SCRIPTS/preprocessing/generate-tfrecord.py \
    -x images/train \
    -l annotations/label_map.pbtxt \
    -o annotations/train.record

# Generate TFRecord for Validation Data
!python $SCRIPTS/preprocessing/generate-tfrecord.py \
    -x images/test \
    -l annotations/label_map.pbtxt \
    -o annotations/test.record

In [ ]:
%cd '/tf/data/MyDrive/tensorflow/power-port-targeting/'
%load_ext tensorboard
%tensorboard --logdir=models/centernet_mobilenetv2_fpn_od

In [ ]:
%cd '/tf/data/MyDrive/tensorflow/power-port-targeting/'
!python /tf/models/research/object_detection/model_main_tf2.py \
    --model_dir=models/centernet_mobilenetv2_fpn_od \
    --pipeline_config_path=models/centernet_mobilenetv2_fpn_od/pipeline.config

In [ ]:
%cd '/tf/data/MyDrive/tensorflow/power-port-targeting/'
!python exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path ./models/centernet_mobilenetv2_fpn_od/pipeline.config \
    --trained_checkpoint_dir ./models/centernet_mobilenetv2_fpn_od/ \
    --output_directory ./exported-models/trained_model

In [ ]:
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL="/tf/data/MyDrive/tensorflow/power-port-targeting/exported-models/trained_model/saved_model"
print('Loading model...', end='')
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)
print('Done!')

category_index=label_map_util.create_category_index_from_labelmap("/tf/data/MyDrive/tensorflow/power-port-targeting/annotations/label_map.pbtxt", use_display_name=True)

img=['/tf/data/MyDrive/tensorflow/power-port-targeting/images/test/frame-321.jpg']
print(img)

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

for image_path in img:
    print('Running inference for {}... '.format(image_path), end='')
    image_np=load_image_into_numpy_array(image_path)
    input_tensor=tf.convert_to_tensor(image_np)
    input_tensor=input_tensor[tf.newaxis, ...]

    detections=detect_fn(input_tensor)num_detections=int(detections.pop('num_detections'))
    detections={key:value[0,:num_detections].numpy() for key,value in detections.items()}
    detections['num_detections']=num_detections
    detections['detection_classes']=detections['detection_classes'].astype(np.int64)
    image_np_with_detections=image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=100,     
          min_score_thresh=.5,      
          agnostic_mode=False)
    %matplotlib inline
    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
    plt.show()